## Basic Python Programming

In [1]:
## Recall the basics
## More to add here.
## Timestamp 
## 2:35:35 to 3:00:00


#Using the print function
print("Hello World")

Hello World


## Building Pac-Man

In [4]:
#Must have board as a py file. opened in the new tab in VS.
from board import boards
import copy
import pygame
import math

pygame.init()

#Dimensions of the screen
WIDTH = 900
HEIGHT = 950
screen = pygame.display.set_mode([WIDTH,HEIGHT])

#Time running into pygame
timer = pygame.time.Clock()
fps = 60
font = pygame.font.Font('freesansbold.ttf',20)
level = copy.deepcopy(boards)

#If we want our future levels to remain red, otherwise change it later.
color = 'red'
PI = math.pi
player_images = []

#This iterates the animation of the player, given the fixed diminensions 
#For example 45 x 45 
#NOTE: ENSURE IT IS IN THE CORRECT DIRECTORY
for i in range(1,5):
    player_images.append(pygame.transform.scale(pygame.image.load(f'player_images/{i}.png'), (45,45)))

#Import the ghost images into the game
blinky_img = pygame.transform.scale(pygame.image.load(f'ghost_images/red.png'), (45,45))
clyde_img = pygame.transform.scale(pygame.image.load(f'ghost_images/orange.png'), (45,45))
pinky_img = pygame.transform.scale(pygame.image.load(f'ghost_images/pink.png'), (45,45))
dead_img = pygame.transform.scale(pygame.image.load(f'ghost_images/dead.png'), (45,45))
spooked_img = pygame.transform.scale(pygame.image.load(f'ghost_images/powerup.png'), (45,45))
inky_img = pygame.transform.scale(pygame.image.load(f'ghost_images/blue.png'), (45,45))

player_x = 450
player_y = 663
direction = 0

# Blinky Ghost initial positions
blinky_x = 56
blinky_y = 58
blinky_direction = 0


# Do Similarly for Clyde, Pinky, and Inky initial positions.
inky_x = 440
inky_y = 388
inky_direction = 2
pinky_x = 440
pinky_y = 438
pinky_direction = 2
clyde_x = 440
clyde_y = 438
clyde_direction = 2



counter = 0
flicker = False

#R, L, U, D
turns_allowed = [False,False,False,False]
direction_command = 0

#You can increase the player speed, whenever you feel like it.
player_speed = 2

#Define the player score
score = 0

#Define power up and eaten ghost
eaten_ghost = [False,False,False,False]
power_counter = 0
powerup=False
startup_counter = 0
targets = [(player_x,player_y),(player_x,player_y),(player_x,player_y),(player_x,player_y)]

#Track whether the ghost is in the box or not
blinky_dead=False
inky_dead = False
clyde_dead = False
pinky_dead = False
blinky_box = False
inky_box = False
clyde_box = False
pinky_box = False

#Ghost Speed you can change later
ghost_speed = [2,2,2,2]
startup_counter = 0


#Number of lives
lives = 3


game_over = False
game_won = False

#Create Class for Ghost
class Ghost:
    def __init__(self, x_coord,y_coord,target,speed,img,direct,dead,box,id):
        self.x_pos = x_coord
        self.y_pos = y_coord
        self.center_x = self.x_pos + 22
        self.center_y = self.y_pos + 22
        self.target = target
        self.speed = speed
        self.img = img
        self.direction = direct
        self.dead = dead
        self.in_box = box
        self.id = id
        self.turns, self.in_box = self.check_collisions()
        self.rect = self.draw()
    
    def draw(self):
        if (not powerup and not self.dead) or (eaten_ghost[self.id] and powerup and not self.dead):
            screen.blit(self.img, (self.x_pos,self.y_pos))
        elif powerup and not self.dead and not eaten_ghost[self.id]:
            screen.blit(spooked_img, (self.x_pos,self.y_pos))
        else:
            screen.blit(dead_img, (self.x_pos,self.y_pos))

        ghost_rect = pygame.rect.Rect((self.center_x - 18, self.center_y - 18), (36,36))
        return ghost_rect
    
    def check_collisions(self):
        # R, L, U, D
        num1 = ((HEIGHT-50)//32)
        num2 = (WIDTH//30)
        num3 = 15
        self.turns = [False,False,False,False]
        if 0 < self.center_x // 30 < 29:
            if level[(self.center_y - num3)//num1][self.center_x//num2] == 9:
                self.turns[2] = True
            if level[self.center_y//num1][(self.center_x-num3)//num2] < 3 \
                or level[self.center_y//num1][(self.center_x-num3)//num2] == 9 and (self.in_box or self.dead):
                self.turns[1] = True

            if level[self.center_y//num1][(self.center_x + num3)//num2] < 3 \
                or level[self.center_y//num1][(self.center_x + num3)//num2] == 9 and (self.in_box or self.dead):
                self.turns[0] = True

            if level[(self.center_y + num3)//num1][self.center_x//num2] < 3 \
                or level[(self.center_y + num3)//num1][self.center_x//num2] == 9 and (self.in_box or self.dead):
                self.turns[3] = True

            if level[(self.center_y - num3)//num1][(self.center_x + num3)//num2] < 3 \
                or level[(self.center_y- num3)//num1][(self.center_x + num3)//num2] == 9 and (self.in_box or self.dead):
                self.turns[2] = True

            if self.direction == 2 or self.direction == 3:
                #For the x-direction
                if 12 <= self.center_x % num2 <= 18:
                    if level[(self.center_y + num3)//num1][self.center_x//num2] < 3 \
                        or (level[(self.center_y + num3)//num1][self.center_x//num2] == 9 and (self.in_box or self.dead)):
                        self.turns[3] = True
                    if level[(self.center_y - num3)//num1][self.center_x//num2] < 3 \
                        or (level[(self.center_y - num3)//num1][self.center_x//num2] == 9 and (self.in_box or self.dead)):
                        self.turns[2] = True   
                #For the y-direction
                if 12 <= self.center_y % num1 <= 18:
                    if level[self.center_y//num1][(self.center_x - num2)//num2] < 3 \
                        or (level[self.center_y//num1][(self.center_x - num2)//num2] == 9 and (self.in_box or self.dead)):
                        self.turns[1] = True
                    if level[self.center_y//num1][(self.center_x + num2)//num2] < 3 \
                        or (level[self.center_y//num1][(self.center_x + num2)//num2] == 9 and (self.in_box or self.dead)):
                        self.turns[0] = True                      

            if self.direction == 0 or self.direction == 1:
                #For the x-direction
                if 12 <= self.center_x % num2 <= 18:
                    if level[(self.center_y + num3)//num1][self.center_x//num2] < 3 \
                        or (level[(self.center_y + num3)//num1][self.center_x//num2] == 9 and (self.in_box or self.dead)):
                        self.turns[3] = True
                    if level[(self.center_y - num3)//num1][self.center_x//num2] < 3 \
                        or (level[(self.center_y - num3)//num1][self.center_x//num2] == 9 and (self.in_box or self.dead)):
                        self.turns[2] = True   
                #For the y-direction
                if 12 <= self.center_y % num1 <= 18:
                    if level[self.center_y//num1][(self.center_x - num3)//num2] < 3 \
                        or (level[self.center_y//num1][(self.center_x - num3)//num2] == 9 and (self.in_box or self.dead)):
                        self.turns[1] = True
                    if level[self.center_y//num1][(self.center_x + num3)//num2] < 3 \
                        or (level[self.center_y//num1][(self.center_x + num3)//num2] == 9 and (self.in_box or self.dead)):
                        self.turns[0] = True   
        else:
            self.turns[0] = True
            self.turns[1] = True
        if 350 < self.x_pos < 550 and 370 < self.y_pos < 480:
            self.in_box = True
        else:
            self.in_box = False
        return self.turns,self.in_box
    
    #Function to able to move ghosts, uniquely 
    def move_clyde(self):
        # r, l, u, d
        # clyde is going to turn whenever advantageous for pursuit
        if self.direction == 0:
            if self.target[0] > self.x_pos and self.turns[0]:
                self.x_pos += self.speed
            elif not self.turns[0]:
                if self.target[1] > self.y_pos and self.turns[3]:
                    self.direction = 3
                    self.y_pos += self.speed
                elif self.target[1] < self.y_pos and self.turns[2]:
                    self.direction = 2
                    self.y_pos -= self.speed
                elif self.target[0] < self.x_pos and self.turns[1]:
                    self.direction = 1
                    self.x_pos -= self.speed
                elif self.turns[3]:
                    self.direction = 3
                    self.y_pos += self.speed
                elif self.turns[2]:
                    self.direction = 2
                    self.y_pos -= self.speed
                elif self.turns[1]:
                    self.direction = 1
                    self.x_pos -= self.speed
            elif self.turns[0]:
                if self.target[1] > self.y_pos and self.turns[3]:
                    self.direction = 3
                    self.y_pos += self.speed
                if self.target[1] < self.y_pos and self.turns[2]:
                    self.direction = 2
                    self.y_pos -= self.speed
                else:
                    self.x_pos += self.speed
        elif self.direction == 1:
            if self.target[1] > self.y_pos and self.turns[3]:
                self.direction = 3
            elif self.target[0] < self.x_pos and self.turns[1]:
                self.x_pos -= self.speed
            elif not self.turns[1]:
                if self.target[1] > self.y_pos and self.turns[3]:
                    self.direction = 3
                    self.y_pos += self.speed
                elif self.target[1] < self.y_pos and self.turns[2]:
                    self.direction = 2
                    self.y_pos -= self.speed
                elif self.target[0] > self.x_pos and self.turns[0]:
                    self.direction = 0
                    self.x_pos += self.speed
                elif self.turns[3]:
                    self.direction = 3
                    self.y_pos += self.speed
                elif self.turns[2]:
                    self.direction = 2
                    self.y_pos -= self.speed
                elif self.turns[0]:
                    self.direction = 0
                    self.x_pos += self.speed
            elif self.turns[1]:
                if self.target[1] > self.y_pos and self.turns[3]:
                    self.direction = 3
                    self.y_pos += self.speed
                if self.target[1] < self.y_pos and self.turns[2]:
                    self.direction = 2
                    self.y_pos -= self.speed
                else:
                    self.x_pos -= self.speed
        elif self.direction == 2:
            if self.target[0] < self.x_pos and self.turns[1]:
                self.direction = 1
                self.x_pos -= self.speed
            elif self.target[1] < self.y_pos and self.turns[2]:
                self.direction = 2
                self.y_pos -= self.speed
            elif not self.turns[2]:
                if self.target[0] > self.x_pos and self.turns[0]:
                    self.direction = 0
                    self.x_pos += self.speed
                elif self.target[0] < self.x_pos and self.turns[1]:
                    self.direction = 1
                    self.x_pos -= self.speed
                elif self.target[1] > self.y_pos and self.turns[3]:
                    self.direction = 3
                    self.y_pos += self.speed
                elif self.turns[1]:
                    self.direction = 1
                    self.x_pos -= self.speed
                elif self.turns[3]:
                    self.direction = 3
                    self.y_pos += self.speed
                elif self.turns[0]:
                    self.direction = 0
                    self.x_pos += self.speed
            elif self.turns[2]:
                if self.target[0] > self.x_pos and self.turns[0]:
                    self.direction = 0
                    self.x_pos += self.speed
                elif self.target[0] < self.x_pos and self.turns[1]:
                    self.direction = 1
                    self.x_pos -= self.speed
                else:
                    self.y_pos -= self.speed
        elif self.direction == 3:
            if self.target[1] > self.y_pos and self.turns[3]:
                self.y_pos += self.speed
            elif not self.turns[3]:
                if self.target[0] > self.x_pos and self.turns[0]:
                    self.direction = 0
                    self.x_pos += self.speed
                elif self.target[0] < self.x_pos and self.turns[1]:
                    self.direction = 1
                    self.x_pos -= self.speed
                elif self.target[1] < self.y_pos and self.turns[2]:
                    self.direction = 2
                    self.y_pos -= self.speed
                elif self.turns[2]:
                    self.direction = 2
                    self.y_pos -= self.speed
                elif self.turns[1]:
                    self.direction = 1
                    self.x_pos -= self.speed
                elif self.turns[0]:
                    self.direction = 0
                    self.x_pos += self.speed
            elif self.turns[3]:
                if self.target[0] > self.x_pos and self.turns[0]:
                    self.direction = 0
                    self.x_pos += self.speed
                elif self.target[0] < self.x_pos and self.turns[1]:
                    self.direction = 1
                    self.x_pos -= self.speed
                else:
                    self.y_pos += self.speed
        if self.x_pos < -30:
            self.x_pos = 900
        elif self.x_pos > 900:
            self.x_pos - 30
        return self.x_pos, self.y_pos, self.direction

    def move_blinky(self):
        # r, l, u, d
        # blinky is going to turn whenever colliding with walls, otherwise continue straight
        if self.direction == 0:
            if self.target[0] > self.x_pos and self.turns[0]:
                self.x_pos += self.speed
            elif not self.turns[0]:
                if self.target[1] > self.y_pos and self.turns[3]:
                    self.direction = 3
                    self.y_pos += self.speed
                elif self.target[1] < self.y_pos and self.turns[2]:
                    self.direction = 2
                    self.y_pos -= self.speed
                elif self.target[0] < self.x_pos and self.turns[1]:
                    self.direction = 1
                    self.x_pos -= self.speed
                elif self.turns[3]:
                    self.direction = 3
                    self.y_pos += self.speed
                elif self.turns[2]:
                    self.direction = 2
                    self.y_pos -= self.speed
                elif self.turns[1]:
                    self.direction = 1
                    self.x_pos -= self.speed
            elif self.turns[0]:
                self.x_pos += self.speed
        elif self.direction == 1:
            if self.target[0] < self.x_pos and self.turns[1]:
                self.x_pos -= self.speed
            elif not self.turns[1]:
                if self.target[1] > self.y_pos and self.turns[3]:
                    self.direction = 3
                    self.y_pos += self.speed
                elif self.target[1] < self.y_pos and self.turns[2]:
                    self.direction = 2
                    self.y_pos -= self.speed
                elif self.target[0] > self.x_pos and self.turns[0]:
                    self.direction = 0
                    self.x_pos += self.speed
                elif self.turns[3]:
                    self.direction = 3
                    self.y_pos += self.speed
                elif self.turns[2]:
                    self.direction = 2
                    self.y_pos -= self.speed
                elif self.turns[0]:
                    self.direction = 0
                    self.x_pos += self.speed
            elif self.turns[1]:
                self.x_pos -= self.speed
        elif self.direction == 2:
            if self.target[1] < self.y_pos and self.turns[2]:
                self.direction = 2
                self.y_pos -= self.speed
            elif not self.turns[2]:
                if self.target[0] > self.x_pos and self.turns[0]:
                    self.direction = 0
                    self.x_pos += self.speed
                elif self.target[0] < self.x_pos and self.turns[1]:
                    self.direction = 1
                    self.x_pos -= self.speed
                elif self.target[1] > self.y_pos and self.turns[3]:
                    self.direction = 3
                    self.y_pos += self.speed
                elif self.turns[3]:
                    self.direction = 3
                    self.y_pos += self.speed
                elif self.turns[0]:
                    self.direction = 0
                    self.x_pos += self.speed
                elif self.turns[1]:
                    self.direction = 1
                    self.x_pos -= self.speed
            elif self.turns[2]:
                self.y_pos -= self.speed
        elif self.direction == 3:
            if self.target[1] > self.y_pos and self.turns[3]:
                self.y_pos += self.speed
            elif not self.turns[3]:
                if self.target[0] > self.x_pos and self.turns[0]:
                    self.direction = 0
                    self.x_pos += self.speed
                elif self.target[0] < self.x_pos and self.turns[1]:
                    self.direction = 1
                    self.x_pos -= self.speed
                elif self.target[1] < self.y_pos and self.turns[2]:
                    self.direction = 2
                    self.y_pos -= self.speed
                elif self.turns[2]:
                    self.direction = 2
                    self.y_pos -= self.speed
                elif self.turns[0]:
                    self.direction = 0
                    self.x_pos += self.speed
                elif self.turns[1]:
                    self.direction = 1
                    self.x_pos -= self.speed
            elif self.turns[3]:
                self.y_pos += self.speed
        if self.x_pos < -30:
            self.x_pos = 900
        elif self.x_pos > 900:
            self.x_pos - 30
        return self.x_pos, self.y_pos, self.direction

    def move_inky(self):
        # r, l, u, d
        # inky turns up or down at any point to pursue, but left and right only on collision
        if self.direction == 0:
            if self.target[0] > self.x_pos and self.turns[0]:
                self.x_pos += self.speed
            elif not self.turns[0]:
                if self.target[1] > self.y_pos and self.turns[3]:
                    self.direction = 3
                    self.y_pos += self.speed
                elif self.target[1] < self.y_pos and self.turns[2]:
                    self.direction = 2
                    self.y_pos -= self.speed
                elif self.target[0] < self.x_pos and self.turns[1]:
                    self.direction = 1
                    self.x_pos -= self.speed
                elif self.turns[3]:
                    self.direction = 3
                    self.y_pos += self.speed
                elif self.turns[2]:
                    self.direction = 2
                    self.y_pos -= self.speed
                elif self.turns[1]:
                    self.direction = 1
                    self.x_pos -= self.speed
            elif self.turns[0]:
                if self.target[1] > self.y_pos and self.turns[3]:
                    self.direction = 3
                    self.y_pos += self.speed
                if self.target[1] < self.y_pos and self.turns[2]:
                    self.direction = 2
                    self.y_pos -= self.speed
                else:
                    self.x_pos += self.speed
        elif self.direction == 1:
            if self.target[1] > self.y_pos and self.turns[3]:
                self.direction = 3
            elif self.target[0] < self.x_pos and self.turns[1]:
                self.x_pos -= self.speed
            elif not self.turns[1]:
                if self.target[1] > self.y_pos and self.turns[3]:
                    self.direction = 3
                    self.y_pos += self.speed
                elif self.target[1] < self.y_pos and self.turns[2]:
                    self.direction = 2
                    self.y_pos -= self.speed
                elif self.target[0] > self.x_pos and self.turns[0]:
                    self.direction = 0
                    self.x_pos += self.speed
                elif self.turns[3]:
                    self.direction = 3
                    self.y_pos += self.speed
                elif self.turns[2]:
                    self.direction = 2
                    self.y_pos -= self.speed
                elif self.turns[0]:
                    self.direction = 0
                    self.x_pos += self.speed
            elif self.turns[1]:
                if self.target[1] > self.y_pos and self.turns[3]:
                    self.direction = 3
                    self.y_pos += self.speed
                if self.target[1] < self.y_pos and self.turns[2]:
                    self.direction = 2
                    self.y_pos -= self.speed
                else:
                    self.x_pos -= self.speed
        elif self.direction == 2:
            if self.target[1] < self.y_pos and self.turns[2]:
                self.direction = 2
                self.y_pos -= self.speed
            elif not self.turns[2]:
                if self.target[0] > self.x_pos and self.turns[0]:
                    self.direction = 0
                    self.x_pos += self.speed
                elif self.target[0] < self.x_pos and self.turns[1]:
                    self.direction = 1
                    self.x_pos -= self.speed
                elif self.target[1] > self.y_pos and self.turns[3]:
                    self.direction = 3
                    self.y_pos += self.speed
                elif self.turns[1]:
                    self.direction = 1
                    self.x_pos -= self.speed
                elif self.turns[3]:
                    self.direction = 3
                    self.y_pos += self.speed
                elif self.turns[0]:
                    self.direction = 0
                    self.x_pos += self.speed
            elif self.turns[2]:
                self.y_pos -= self.speed
        elif self.direction == 3:
            if self.target[1] > self.y_pos and self.turns[3]:
                self.y_pos += self.speed
            elif not self.turns[3]:
                if self.target[0] > self.x_pos and self.turns[0]:
                    self.direction = 0
                    self.x_pos += self.speed
                elif self.target[0] < self.x_pos and self.turns[1]:
                    self.direction = 1
                    self.x_pos -= self.speed
                elif self.target[1] < self.y_pos and self.turns[2]:
                    self.direction = 2
                    self.y_pos -= self.speed
                elif self.turns[2]:
                    self.direction = 2
                    self.y_pos -= self.speed
                elif self.turns[1]:
                    self.direction = 1
                    self.x_pos -= self.speed
                elif self.turns[0]:
                    self.direction = 0
                    self.x_pos += self.speed
            elif self.turns[3]:
                self.y_pos += self.speed
        if self.x_pos < -30:
            self.x_pos = 900
        elif self.x_pos > 900:
            self.x_pos - 30
        return self.x_pos, self.y_pos, self.direction

    def move_pinky(self):
        # r, l, u, d
        # inky is going to turn left or right whenever advantageous, but only up or down on collision
        if self.direction == 0:
            if self.target[0] > self.x_pos and self.turns[0]:
                self.x_pos += self.speed
            elif not self.turns[0]:
                if self.target[1] > self.y_pos and self.turns[3]:
                    self.direction = 3
                    self.y_pos += self.speed
                elif self.target[1] < self.y_pos and self.turns[2]:
                    self.direction = 2
                    self.y_pos -= self.speed
                elif self.target[0] < self.x_pos and self.turns[1]:
                    self.direction = 1
                    self.x_pos -= self.speed
                elif self.turns[3]:
                    self.direction = 3
                    self.y_pos += self.speed
                elif self.turns[2]:
                    self.direction = 2
                    self.y_pos -= self.speed
                elif self.turns[1]:
                    self.direction = 1
                    self.x_pos -= self.speed
            elif self.turns[0]:
                self.x_pos += self.speed
        elif self.direction == 1:
            if self.target[1] > self.y_pos and self.turns[3]:
                self.direction = 3
            elif self.target[0] < self.x_pos and self.turns[1]:
                self.x_pos -= self.speed
            elif not self.turns[1]:
                if self.target[1] > self.y_pos and self.turns[3]:
                    self.direction = 3
                    self.y_pos += self.speed
                elif self.target[1] < self.y_pos and self.turns[2]:
                    self.direction = 2
                    self.y_pos -= self.speed
                elif self.target[0] > self.x_pos and self.turns[0]:
                    self.direction = 0
                    self.x_pos += self.speed
                elif self.turns[3]:
                    self.direction = 3
                    self.y_pos += self.speed
                elif self.turns[2]:
                    self.direction = 2
                    self.y_pos -= self.speed
                elif self.turns[0]:
                    self.direction = 0
                    self.x_pos += self.speed
            elif self.turns[1]:
                self.x_pos -= self.speed
        elif self.direction == 2:
            if self.target[0] < self.x_pos and self.turns[1]:
                self.direction = 1
                self.x_pos -= self.speed
            elif self.target[1] < self.y_pos and self.turns[2]:
                self.direction = 2
                self.y_pos -= self.speed
            elif not self.turns[2]:
                if self.target[0] > self.x_pos and self.turns[0]:
                    self.direction = 0
                    self.x_pos += self.speed
                elif self.target[0] < self.x_pos and self.turns[1]:
                    self.direction = 1
                    self.x_pos -= self.speed
                elif self.target[1] > self.y_pos and self.turns[3]:
                    self.direction = 3
                    self.y_pos += self.speed
                elif self.turns[1]:
                    self.direction = 1
                    self.x_pos -= self.speed
                elif self.turns[3]:
                    self.direction = 3
                    self.y_pos += self.speed
                elif self.turns[0]:
                    self.direction = 0
                    self.x_pos += self.speed
            elif self.turns[2]:
                if self.target[0] > self.x_pos and self.turns[0]:
                    self.direction = 0
                    self.x_pos += self.speed
                elif self.target[0] < self.x_pos and self.turns[1]:
                    self.direction = 1
                    self.x_pos -= self.speed
                else:
                    self.y_pos -= self.speed
        elif self.direction == 3:
            if self.target[1] > self.y_pos and self.turns[3]:
                self.y_pos += self.speed
            elif not self.turns[3]:
                if self.target[0] > self.x_pos and self.turns[0]:
                    self.direction = 0
                    self.x_pos += self.speed
                elif self.target[0] < self.x_pos and self.turns[1]:
                    self.direction = 1
                    self.x_pos -= self.speed
                elif self.target[1] < self.y_pos and self.turns[2]:
                    self.direction = 2
                    self.y_pos -= self.speed
                elif self.turns[2]:
                    self.direction = 2
                    self.y_pos -= self.speed
                elif self.turns[1]:
                    self.direction = 1
                    self.x_pos -= self.speed
                elif self.turns[0]:
                    self.direction = 0
                    self.x_pos += self.speed
            elif self.turns[3]:
                if self.target[0] > self.x_pos and self.turns[0]:
                    self.direction = 0
                    self.x_pos += self.speed
                elif self.target[0] < self.x_pos and self.turns[1]:
                    self.direction = 1
                    self.x_pos -= self.speed
                else:
                    self.y_pos += self.speed
        if self.x_pos < -30:
            self.x_pos = 900
        elif self.x_pos > 900:
            self.x_pos - 30
        return self.x_pos, self.y_pos, self.direction


#For this function draw_misc
# We want to display the Player Score
def draw_misc():
    score_text = font.render(f'Score: {score}',True,'white')
    screen.blit(score_text,(10,920))
    if powerup:
        pygame.draw.circle(screen,'blue',(140,930),15)
    for i in range(lives):
        screen.blit(pygame.transform.scale(player_images[0],(30,30)),(650 + i* 40, 915))
    if game_over:
        pygame.draw.rect(screen,'white',[50,200,800,300],0,10)
        pygame.draw.rect(screen,'dark grey',[70,220,760,326],0,10)
        game_over_text = font.render('Game over! Space bar to restart!',True,'red')
        screen.blit(game_over_text, (100,300))
    if game_won:
        pygame.draw.rect(screen,'white',[50,200,800,300],0,10)
        pygame.draw.rect(screen,'dark grey',[70,220,760,326],0,10)
        game_over_text = font.render('Victory! Space bar to restart!',True,'green')
        screen.blit(game_over_text, (100,300))


#For this function check_collison
#We want to make sure the player makes contact on a certain object
def check_collisions(scor, power, power_count,eaten_ghosts):
    num1 = (HEIGHT - 50)//32
    num2 = WIDTH//30
    if 0 < player_x <870:
        if level[center_y//num1][center_x//num2] == 1:
            level[center_y//num1][center_x//num2] =0
            scor += 10
        if level[center_y//num1][center_x//num2] == 2:
            level[center_y//num1][center_x//num2] = 0
            scor += 50
            power = True
            power_count = 0
            eaten_ghosts = [False,False,False]
    return scor, power, power_count,eaten_ghosts


#For this function draw_board
#We want to visualise how the pac-man-board works into Pygame.
def draw_board():
    #num1 and num2 must be integers, reasons!
    num1 = ((HEIGHT - 50)//32)
    num2 = (WIDTH //30)
    for i in range(len(level)):
        for j in range(len(level[i])):
            if level[i][j] == 1:
                # We wanted the Pac-man to eat the white circles
                pygame.draw.circle(screen,'white',(j*num2 + (0.5*num2), i*num1 + (0.5*num1)),4)
            if level[i][j] == 2 and not flicker:
                pygame.draw.circle(screen,'white',(j*num2 + (0.5*num2), i*num1 + (0.5*num1)),10)
            if level[i][j] == 3:
                pygame.draw.line(screen,color,(j*num2 + (0.5*num2), i*num1),
                                   (j*num2 + (0.5*num2), i*num1 + num1),3)
            if level[i][j] == 4:
                pygame.draw.line(screen,color,(j*num2, i*num1 + (0.5*num1)),
                                   (j*num2 + num2, i*num1 + (0.5*num1)),3)
            if level[i][j] == 5:
                pygame.draw.arc(screen,color,[(j*num2 - (num2*0.4)) -2, (i*num1 + (0.5*num1)),num2,num1],0,PI/2,3)

            if level[i][j] == 6:
                pygame.draw.arc(screen,color,[(j*num2 + (num2*0.5)), (i*num1 + (0.5*num1)),num2,num1],PI/2,PI,3)
            if level[i][j] == 7:
                pygame.draw.arc(screen,color,[(j*num2 + (num2*0.5)), (i*num1 - (0.4*num1)),num2,num1],PI,3*PI/2,3)

            if level[i][j] == 8:
                pygame.draw.arc(screen,color,[(j*num2 - (num2*0.4)) - 2, (i*num1 - (0.4*num1)),num2,num1],3*PI/2,2*PI,3)

            if level[i][j] == 9:
                #Displays the ghost border at the centre of the board
                pygame.draw.line(screen,'white',(j*num2, i*num1 + (0.5*num1)),
                                   (j*num2 + num2, i*num1 + (0.5*num1)),3)


#For this function draw_player
#We want to visualise the character pac-man (the player) into Pygame.
def draw_player():
    # Directions are 
    # 0-RIGHT
    # 1-LEFT
    # 2-UP
    # 3-DOWN respectively in that order.
    if direction == 0:
        screen.blit(player_images[counter//5], (player_x, player_y))
    if direction == 1:
        screen.blit(pygame.transform.flip(player_images[counter//5],True,False), (player_x, player_y))
    if direction == 2:
        screen.blit(pygame.transform.rotate(player_images[counter//5],90), (player_x, player_y))
    if direction == 3:
        # Note: Exercise see if you change it to -90!
        screen.blit(pygame.transform.rotate(player_images[counter//5],270), (player_x, player_y))


#For this function check_position 
#We wanted to understand the character pac-man (the player) position into Pygame.
def check_position(centerx,centery):
    turns = [False,False,False,False]
    num1 = (HEIGHT-50)//32
    num2 = (WIDTH//30)
    num3 = 15
    #check collisions based on center x and center y of player
    if centerx//30 < 29:
        if direction == 0:
            if level[centery//num1][(centerx - num3)//num2] <3:
                turns[1] = True
        if direction == 1:
            if level[centery//num1][(centerx + num3)//num2] <3:
                turns[0] = True
        if direction == 2:
            if level[(centery+num3)//num1][centerx//num2] <3:
                turns[3] = True
        if direction == 3:
            if level[(centery-num3)//num1][centerx//num2] <3:
                turns[2] = True
        if direction == 2 or direction == 3:
            if 12 <= centerx % num2 <= 18:
                if level[(centery + num3)//num1][centerx //num2] < 3:
                    turns[3] = True
                if level[(centery - num3)//num1][centerx //num2] < 3:
                    turns[2] = True
            if 12 <= centery % num1 <= 18:
                if level[centery//num1][(centerx - num2) //num2] < 3:
                    turns[1] = True
                if level[centery//num1][(centerx + num2) //num2] < 3:
                    turns[0] = True
        if direction == 0 or direction == 1:
            if 12 <= centerx % num2 <= 18:
                if level[(centery + num1)//num1][centerx //num2] < 3:
                    turns[3] = True
                if level[(centery - num1)//num1][centerx //num2] < 3:
                    turns[2] = True
            if 12 <= centery % num1 <= 18:
                if level[centery//num1][(centerx - num3) //num2] < 3:
                    turns[1] = True
                if level[centery//num1][(centerx + num3) //num2] < 3:
                    turns[0] = True
    else:
        turns[0] = True
        turns[1] = True
    return turns


def move_player(play_x,play_y):
    #r,l,u,d
    if direction == 0 and turns_allowed[0]:
        play_x += player_speed
    elif direction == 1 and turns_allowed[1]:
        play_x -= player_speed
    if direction == 2 and turns_allowed[2]:
        play_y -= player_speed
    elif direction == 3 and turns_allowed[3]:
        play_y += player_speed     
    return play_x,play_y

def get_targets(blink_x, blink_y, ink_x, ink_y, pink_x, pink_y, clyd_x, clyd_y):
    if player_x < 450:
        runaway_x = 900
    else:
        runaway_x = 0
    if player_y < 450:
        runaway_y = 900
    else:
        runaway_y = 0

    return_target = (380, 400)
    if powerup:
        if not blinky.dead and not eaten_ghost[0]:
            blink_target = (runaway_x, runaway_y)
        elif not blinky.dead and eaten_ghost[0]:
            if 340 < blink_x < 560 and 340 < blink_y < 500:
                blink_target = (400, 100)
            else:
                blink_target = (player_x, player_y)
        else:
            blink_target = return_target
        if not inky.dead and not eaten_ghost[1]:
            ink_target = (runaway_x, player_y)
        elif not inky.dead and eaten_ghost[1]:
            if 340 < ink_x < 560 and 340 < ink_y < 500:
                ink_target = (400, 100)
            else:
                ink_target = (player_x, player_y)
        else:
            ink_target = return_target
        if not pinky.dead:
            pink_target = (player_x, runaway_y)
        elif not pinky.dead and eaten_ghost[2]:
            if 340 < pink_x < 560 and 340 < pink_y < 500:
                pink_target = (400, 100)
            else:
                pink_target = (player_x, player_y)
        else:
            pink_target = return_target
        if not clyde.dead and not eaten_ghost[-1]:
            clyd_target = (450, 450)
        elif not clyde.dead and eaten_ghost[-1]:
            if 340 < clyd_x < 560 and 340 < clyd_y < 500:
                clyd_target = (400, 100)
            else:
                clyd_target = (player_x, player_y)
        else:
            clyd_target = return_target
    else:
        if not blinky.dead:
            if 340 < blink_x < 560 and 340 < blink_y < 500:
                blink_target = (400, 100)
            else:
                blink_target = (player_x, player_y)
        else:
            blink_target = return_target

        if not inky.dead:
            if 340 < ink_x < 560 and 340 < ink_y < 500:
                ink_target = (400, 100)
            else:
                ink_target = (player_x, player_y)
        else:
            ink_target = return_target
        if not pinky.dead:
            if 340 < pink_x < 560 and 340 < pink_y < 500:
                pink_target = (400, 100)
            else:
                pink_target = (player_x, player_y)
        else:
            pink_target = return_target
        if not clyde.dead:
            if 340 < clyd_x < 560 and 340 < clyd_y < 500:
                clyd_target = (400, 100)
            else:
                clyd_target = (player_x, player_y)
        else:
            clyd_target = return_target
    return [blink_target, ink_target, pink_target, clyd_target]


#This opens up the Pygame Window 
run = True
while run:
    timer.tick(fps)
    # Speed for the player animation
    if counter < 19:
        counter+= 1
        if counter > 3:
            flicker = False
    else:
        # For the big white circles to flicker (on and off)
        # may vary on the counter (you can adjust).
        counter = 0
        flicker = True
    if powerup and power_counter < 600:
        power_counter += 1
    elif powerup and power_counter >= 600:
        power_counter = 0
        powerup = False
        eaten_ghost = [False,False,False,False]
    if startup_counter < 180 and not game_over and not game_won:
        moving = False
        startup_counter += 1
    else:
        moving = True

    screen.fill('black')
    draw_board()
    center_x = player_x + 23
    center_y = player_y + 24
    if powerup:
        ghost_speed = [1,1,1,1]
    else:
        ghost_speed = [2,2,2,2]
    if eaten_ghost[0]:
        ghost_speed[0] = 2
    if eaten_ghost[1]:
        ghost_speed[1] = 2
    if eaten_ghost[2]:
        ghost_speed[2] = 2
    if eaten_ghost[-1]:
        ghost_speed[-1] = 2
    if blinky_dead:
        ghost_speed[0] = 4
    if inky_dead:
        ghost_speed[1] = 4
    if pinky_dead:
        ghost_speed[2] = 4
    if clyde_dead:
        ghost_speed[3] = 4

    game_won = True
    for i in range(len(level)):
        if 1 in level[i] or 2 in level[i]:
            game_won = False




    player_circle = pygame.draw.circle(screen,'black',(center_x,center_y),21,2)
    draw_player()

    #Calling the functions for the ghosts into Pygame
    blinky = Ghost(blinky_x, blinky_y, targets[0], ghost_speed[0], blinky_img, blinky_direction, blinky_dead,
                   blinky_box, 0)
    inky = Ghost(inky_x,inky_y,targets[1],ghost_speed[1], inky_img, inky_direction, inky_dead, inky_box, 1)
    pinky = Ghost(pinky_x,pinky_y,targets[2],ghost_speed[2], pinky_img, pinky_direction, pinky_dead, pinky_box, 2)
    clyde = Ghost(clyde_x,clyde_y,targets[-1],ghost_speed[-1], clyde_img, clyde_direction, clyde_dead, clyde_box, -1)

    draw_misc()
    targets = get_targets(blinky_x,blinky_y,inky_x,inky_y,pinky_x,pinky_y, clyde_x,clyde_y)
    pygame.draw.circle(screen,'white',(center_x,center_y),2)
    turns_allowed = check_position(center_x,center_y)
    if moving:
        player_x,player_y = move_player(player_x,player_y)
        if not blinky_dead and not blinky.in_box:
            blinky_x, blinky_y, blinky_direction = blinky.move_blinky()
        else:
            blinky_x,blinky_y,blinky_direction = blinky.move_clyde()

        if not pinky_dead and not pinky.in_box:
            pinky_x,pinky_y,pinky_direction = pinky.move_pinky()
        else:
            pinky_x,pinky_y,pinky_direction = pinky.move_clyde()

        if not inky_dead and not inky.in_box:
            inky_x,inky_y,inky_direction = inky.move_inky()
        else:
            inky_x,inky_y,inky_direction = inky.move_clyde()
        clyde_x,clyde_y, clyde_direction = clyde.move_clyde()
    score, powerup, power_counter, eaten_ghost = check_collisions(score,powerup, power_counter,eaten_ghost)

    #To check the collison if the player hits the ghost
    # Add to if not powerup to check if eaten ghosts
    if not powerup:
        if (player_circle.colliderect(blinky.rect) and not blinky.dead) or \
        (player_circle.colliderect(inky.rect) and not inky.dead) or \
        (player_circle.colliderect(pinky.rect) and not pinky.dead) or \
        (player_circle.colliderect(clyde.rect) and not clyde.dead):
            if lives > 0:
                powerup = False
                power_counter = 0
                lives -= 1
                startup_counter = 0
                player_x = 450
                player_y = 663
                direction = 0
                direction_command = 0
                blinky_x = 56
                blinky_y = 58
                blinky_direction = 0
                inky_x = 440
                inky_y = 388
                inky_direction = 2
                pinky_x = 440
                pinky_y = 438
                pinky_direction = 2
                clyde_x = 440
                clyde_y = 438
                clyde_direction = 2
                eaten_ghost = [False, False, False, False]
                blinky_dead = False
                inky_dead = False
                clyde_dead = False
                pinky_dead = False  
            else:
                game_over = True
                moving = False
                startup_counter = 0

    if powerup and player_circle.colliderect(blinky.rect) and eaten_ghost[0] and not blinky.dead: 
        if lives > 0:
            powerup = False
            power_counter = 0
            lives -= 1
            startup_counter = 0
            player_x = 450
            player_y = 663
            direction = 0
            direction_command = 0
            blinky_x = 56
            blinky_y = 58
            blinky_direction = 0
            inky_x = 440
            inky_y = 388
            inky_direction = 2
            pinky_x = 440
            pinky_y = 438
            pinky_direction = 2
            clyde_x = 440
            clyde_y = 438
            clyde_direction = 2
            eaten_ghost = [False, False, False, False]
            blinky_dead = False
            inky_dead = False
            clyde_dead = False
            pinky_dead = False
        else:
            game_over = True
            moving = False
            startup_counter = 0
    if powerup and player_circle.colliderect(inky.rect) and eaten_ghost[1] and not inky.dead:
        if lives > 0:
            powerup = False
            power_counter = 0
            lives -= 1
            startup_counter = 0
            player_x = 450
            player_y = 663
            direction = 0
            direction_command = 0
            blinky_x = 56
            blinky_y = 58
            blinky_direction = 0
            inky_x = 440
            inky_y = 388
            inky_direction = 2
            pinky_x = 440
            pinky_y = 438
            pinky_direction = 2
            clyde_x = 440
            clyde_y = 438
            clyde_direction = 2
            eaten_ghost = [False, False, False, False]
            blinky_dead = False
            inky_dead = False
            clyde_dead = False
            pinky_dead = False
        else:
            game_over = True
            moving = False
            startup_counter = 0
    if powerup and player_circle.colliderect(pinky.rect) and eaten_ghost[2] and not pinky.dead:
        if lives > 0:
            powerup = False
            power_counter = 0
            lives -= 1
            startup_counter = 0
            player_x = 450
            player_y = 663
            direction = 0
            direction_command = 0
            blinky_x = 56
            blinky_y = 58
            blinky_direction = 0
            inky_x = 440
            inky_y = 388
            inky_direction = 2
            pinky_x = 440
            pinky_y = 438
            pinky_direction = 2
            clyde_x = 440
            clyde_y = 438
            clyde_direction = 2
            eaten_ghost = [False, False, False, False]
            blinky_dead = False
            inky_dead = False
            clyde_dead = False
            pinky_dead = False
        else:
            game_over = True
            moving = False
            startup_counter = 0
    if powerup and player_circle.colliderect(clyde.rect) and eaten_ghost[-1] and not clyde.dead:
        if lives > 0:
            powerup = False
            power_counter = 0
            lives -= 1
            startup_counter = 0
            player_x = 450
            player_y = 663
            direction = 0
            direction_command = 0
            blinky_x = 56
            blinky_y = 58
            blinky_direction = 0
            inky_x = 440
            inky_y = 388
            inky_direction = 2
            pinky_x = 440
            pinky_y = 438
            pinky_direction = 2
            clyde_x = 440
            clyde_y = 438
            clyde_direction = 2
            eaten_ghost = [False, False, False, False]
            blinky_dead = False
            inky_dead = False
            clyde_dead = False
            pinky_dead = False
        else:
            game_over = True
            moving = False
            startup_counter = 0

    if powerup and player_circle.colliderect(blinky.rect) and not blinky.dead and not eaten_ghost[0]:
        blinky_dead=True
        eaten_ghost[0] = True
        score += (2 ** eaten_ghost.count(True)) * 100
    if powerup and player_circle.colliderect(inky.rect) and not inky.dead and not eaten_ghost[1]:
        inky_dead=True
        eaten_ghost[1] = True
        score += (2 ** eaten_ghost.count(True)) * 100
    if powerup and player_circle.colliderect(pinky.rect) and not pinky.dead and not eaten_ghost[2]:
        pinky_dead=True
        eaten_ghost[2] = True
        score += (2 ** eaten_ghost.count(True)) * 100
    if powerup and player_circle.colliderect(clyde.rect) and not clyde.dead and not eaten_ghost[-1]:
        clyde_dead=True
        eaten_ghost[-1] = True
        score += (2 ** eaten_ghost.count(True)) * 100
       


    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            run = False
        #Controls the player
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_RIGHT:
                direction_command = 0
            if event.key == pygame.K_LEFT:
                direction_command = 1
            if event.key == pygame.K_UP:
                direction_command = 2
            if event.key == pygame.K_DOWN:
                direction_command = 3
            if event.key == pygame.K_SPACE and (game_over or game_won):
                powerup = False
                power_counter = 0
                lives -= 1
                startup_counter = 0
                player_x = 450
                player_y = 663
                direction = 0
                direction_command = 0
                blinky_x = 56
                blinky_y = 58
                blinky_direction = 0
                inky_x = 440
                inky_y = 388
                inky_direction = 2
                pinky_x = 440
                pinky_y = 438
                pinky_direction = 2
                clyde_x = 440
                clyde_y = 438
                clyde_direction = 2
                eaten_ghost = [False, False, False, False]
                blinky_dead = False
                inky_dead = False
                clyde_dead = False
                pinky_dead = False
                score = 0
                #You can change the number of lives
                lives = 3
                #After game won or over the board should go the way exactly the way it is!
                level = copy.deepcopy(boards)
                game_over = False
                game_won = False
            
        if event.type == pygame.KEYUP and direction_command == 0:
            if event.key == pygame.K_RIGHT:
                direction_command = direction
            if event.key == pygame.K_LEFT and direction_command == 1:
                direction_command = direction
            if event.key == pygame.K_UP and direction_command == 2:
                direction_command = direction
            if event.key == pygame.K_DOWN and direction_command == 3:
                direction_command = direction
        

        if direction_command == 0 and turns_allowed[0]:
            direction = 0
        if direction_command == 1 and turns_allowed[1]:
            direction = 1
        if direction_command == 2 and turns_allowed[2]:
            direction = 2
        if direction_command == 3 and turns_allowed[3]:
            direction = 3
        
        # Teleports Pac-man (the player) to the opposite side of the screen
        # depending on either side of the board.
        if player_x > 900:
            player_x = - 47
        elif player_x < -50:
            player_x = 897

        # Reset if the Ghost is dead when they enter ghost box
        if blinky.in_box and blinky_dead:
            blinky_dead = False
        
        if inky.in_box and inky_dead:
            inky_dead = False

        if pinky.in_box and pinky_dead:
            picky_dead = False

        if clyde.in_box and clyde_dead:
            clyde_dead = False
    

    pygame.display.flip()

pygame.quit()

### Credit to LeMaster Tech, I will continue to watch some Youtube tutorials on using Python's package Pygame, If I have the time.